## Device use T4 * 2

#### Clone WMamba & U-Mamba Repo

In [1]:
!git clone https://github.com/Yui-Arthur/WMamba.git
%cd WMamba
!git clone https://github.com/bowang-lab/U-Mamba.git

Cloning into 'WMamba'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 106 (delta 16), reused 15 (delta 8), pack-reused 68
Receiving objects: 100% (106/106), 73.54 KiB | 1.63 MiB/s, done.
Resolving deltas: 100% (47/47), done.
/kaggle/working/WMamba
Cloning into 'U-Mamba'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 301 (delta 46), reused 18 (delta 16), pack-reused 231
Receiving objects: 100% (301/301), 2.91 MiB | 15.54 MiB/s, done.
Resolving deltas: 100% (74/74), done.


#### Mabma & U-Mamba & WMamba reqirement

In [2]:
!pip install causal-conv1d>=1.2.0 --no-cache-dir > /dev/null
!pip install mamba-ssm --no-cache-dir > /dev/null
!cd U-Mamba/umamba && (pip install -e .) > /dev/null

In [3]:
!pip install ptwt >  /dev/null
!pip install PyWavelets >  /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.1.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [4]:
!sudo bash setup_wmamba.sh

Setup WMamba
CP File From W-Mamba to U-Mamba
Trainer:
  nnUNetTrainerUMambaEnc.py
  nnUNetTrainerUMambaBot.py
  nnUNetTrainerWMambaBase.py
  nnUNetTrainerWMambaBot.py
  nnUNetTrainerUMambaEncNoAMP.py
Nets:
  WMambaBot_3d.py
Others:
  paths.py


### set env variable

In [5]:
### DO NOT EDIT !!! ###
nnUNet_preprocessed="/home/nnUNet_preprocessed/"
nnUNet_raw="/home/nnUNet_raw/"
nnUNet_results="/kaggle/working/"

%env nnUNet_preprocessed={nnUNet_preprocessed}
%env nnUNet_raw={nnUNet_raw}
%env nnUNet_results={nnUNet_results}



env: nnUNet_preprocessed=/home/nnUNet_preprocessed/
env: nnUNet_raw=/home/nnUNet_raw/
env: nnUNet_results=/kaggle/working/


### parameter (can modify)

In [6]:
### Epoch ###
# **the maximum epoch include all train epoch in every session** #
%env training_epochs=150

### Checkpoint ###
# every {checkpoint_save_every} epochs will save one checkpoint
%env checkpoint_save_every = 10
load_checkpoint = True
prev_checkpoint_f0 = "/kaggle/input/0429_final/checkpoint_best_0.pth"
prev_checkpoint_f1 = "/kaggle/input/0429_final/checkpoint_best_1.pth"

### Kaggle Dataset Input Path ###
dataset_path = "/kaggle/input/Dataset110_CTImage.mp4"
### Dataset ID ###
dataset_id = 110

### Set Trainer/Model ###
trainer = "nnUNetTrainerUMambaBot"


env: training_epochs=150
env: checkpoint_save_every=10


### dataset prepaired

In [7]:
if dataset_id == 110:
    """===== Dataset 110 Covid-19 ===="""
    """ unrar data folder and move to home. """
    !apt install unrar
    !(unrar x {dataset_path} /home/nnUNet_raw/) >  /dev/null
    """dataset preoproccess about 20 mins."""
    !nnUNetv2_plan_and_preprocess -d 110 --verify_dataset_integrity -c 3d_fullres
    dataset_id = 110
    dataset_name = "Dataset110_CTImage"
else:
    """===== Dastaset 4 Hippocampus ===="""
    """ unzip data folder and move to home. """
    !(tar -C  /home/ -xvf {dataset_path}) > /dev/null
    """ dataset preoproccess. """
    !nnUNetv2_convert_MSD_dataset -i /home/Task04_Hippocampus
    !nnUNetv2_plan_and_preprocess -d 4 --verify_dataset_integrity -c 3d_fullres
    dataset_id = 4
    dataset_name = "Dataset004_Hippocampus"





The following NEW packages will be installed:
  unrar
0 upgraded, 1 newly installed, 0 to remove and 51 not upgraded.
Need to get 113 kB of archives.
After this operation, 406 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 unrar amd64 1:5.6.6-2build1 [113 kB]
Fetched 113 kB in 0s (509 kB/s)

78Selecting previously unselected package unrar.
(Reading database ... 113807 files and directories currently installed.)
Preparing to unpack .../unrar_1%3a5.6.6-2build1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking unrar (1:5.6.6-2build1) ...
7Progress: [ 40%] [#######################...................................] 8Setting up unrar (1:5.6.6-2build1) ...
7Progress: [ 60%] [##################################........................] 8update-alternatives: using /usr/bin/unrar-nonfree to pro

### train

In [8]:
from pathlib import Path
if not load_checkpoint:
    ### first training (no checkpoint) ###
    !CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr {trainer} --npz & \
    CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr {trainer} --npz
else:
    ### load checkpoint, continue train
    current_trainer_folder = f"{nnUNet_results}{dataset_name}/{trainer}__nnUNetPlans__3d_fullres/"

    !mkdir -p {current_trainer_folder}/fold_0
    !mkdir -p {current_trainer_folder}/fold_1

    !cp {prev_checkpoint_f0} {current_trainer_folder}/fold_0/checkpoint_best.pth
    !cp {prev_checkpoint_f1} {current_trainer_folder}/fold_1/checkpoint_best.pth
    
    # check the checkpoint can be successful loaded
    f0 = [i.name for i in Path(current_trainer_folder+"fold_0").glob("*.pth")]
    f1 = [i.name for i in Path(current_trainer_folder+"fold_1").glob("*.pth")]
    print("Load Checkpoint fold 0",f0)
    print("Load Checkpoint fold 1",f1)
    if len(f0) <1 or len(f1) <1:
        raise ValueError("Load Checkpoint Failed")
    
    !CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr {trainer} --c --npz & \
    CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr {trainer} --c  --npz

Load Checkpoint fold 0 ['checkpoint_best.pth']
Load Checkpoint fold 1 ['checkpoint_best.pth']
2024-04-30 15:39:36.737937: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 15:39:36.737955: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 15:39:36.738003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 15:39:36.738062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 15:39:37.012787: E external/local_xla/xla/stream_exec

### find best fold

In [9]:
!nnUNetv2_find_best_configuration {dataset_id} -c 3d_fullres -tr nnUNetTrainerWMambaBot -f 0 1 > /dev/null

Traceback (most recent call last):
  File "/opt/conda/bin/nnUNetv2_find_best_configuration", line 33, in <module>
    sys.exit(load_entry_point('nnunetv2', 'console_scripts', 'nnUNetv2_find_best_configuration')())
  File "/kaggle/working/WMamba/U-Mamba/umamba/nnunetv2/evaluation/find_best_configuration.py", line 295, in find_best_configuration_entry_point
    find_best_configuration(dataset_name, model_dict, allow_ensembling=not args.disable_ensembling,
  File "/kaggle/working/WMamba/U-Mamba/umamba/nnunetv2/evaluation/find_best_configuration.py", line 91, in find_best_configuration
    allowed_trained_models = filter_available_models(deepcopy(allowed_trained_models), dataset_name_or_id)
  File "/kaggle/working/WMamba/U-Mamba/umamba/nnunetv2/evaluation/find_best_configuration.py", line 43, in filter_available_models
    raise RuntimeError(f"Trained model {trained_model} does not have an output folder. "
RuntimeError: Trained model {'plans': 'nnUNetPlans', 'configuration': '3d_fullres', 

### predict 

In [10]:
!nnUNetv2_predict -i "{nnUNet_raw}{dataset_name}/imagesTs"\
-o "{nnUNet_results}{dataset_name}/predTs" \
-d {dataset_id} -c 3d_fullres -tr {trainer} -f 0 1 > /dev/null

2024-04-30 17:00:30.733327: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 17:00:30.733469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 17:00:30.946989: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|███████████████████████████████████████████| 27/27 [00:54<00:00,  2.03s/it]


In [11]:
####
!python ./createSubmissionFile.py -s "{nnUNet_results}{dataset_name}/predTs" -d "{nnUNet_results}/predTs_rename"
####
!tar zcvf pred.tar.gz "{nnUNet_results}/predTs_rename"

[01/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0108.nii.gz to /kaggle/working//predTs_rename/108.nii.gz
[02/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0451.nii.gz to /kaggle/working//predTs_rename/451.nii.gz
[03/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0229.nii.gz to /kaggle/working//predTs_rename/229.nii.gz
[04/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0064.nii.gz to /kaggle/working//predTs_rename/064.nii.gz
[05/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0257.nii.gz to /kaggle/working//predTs_rename/257.nii.gz
[06/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0523.nii.gz to /kaggle/working//predTs_rename/523.nii.gz
[07/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0156.nii.gz to /kaggle/working//predTs_rename/156.nii.gz
[08/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/c